In [29]:
from scrapfly import ScrapflyClient, ScrapeConfig, ScrapflyError
 
scrapfly = ScrapflyClient(key="scp-live-01fc8534987042f4a2fe553c5cf6a2df", max_concurrency=2)
url = 'https://www.samsclub.com/p/P03020955'
listing_result = scrapfly.scrape(ScrapeConfig(url=url, render_js=True, country="US", asp=True, retry=False, rendering_wait=10000))
listing_html = listing_result.scrape_result['content']

In [30]:
from bs4 import BeautifulSoup
import re
import json
def extract_json_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    script_tags = soup.find_all('script')

    json_data_list = []

    for script in script_tags:
        script_content = script.string
        if script_content:
            # Use a regular expression to find JSON-like content
            json_pattern = re.compile(r'({.*})', re.DOTALL)
            matches = json_pattern.findall(script_content)
            for match in matches:
                try:
                    json_data = json.loads(match)
                    json_data_list.append(json_data)
                except json.JSONDecodeError:
                    pass  # Ignore invalid JSON

    return json_data_list

json_data_list = extract_json_from_html(listing_html)

In [33]:
data = json_data_list[2]['cache']['products']

In [60]:
product = data['P03020955']

output = {
    'productId': product.get('productId'),
    'productType': product.get('productType'),
    'productDescription': product.get('descriptors', {}).get('productdescription'),
    'shortDescription': product.get('descriptors', {}).get('shortDescription'),
    'longDescription': product.get('descriptors', {}).get('longDescription'),
    'categoryDisplayName': product.get('category', {}).get('breadcrumbs', [{}])[-1].get('displayName'),
    'url': product.get('searchAndSeo', {}).get('url'),
    'shippingOption': product.get('shippingOption', {}).get('info'),
    'model': product.get('manufacturingInfo', {}).get('model'),
    'brand': product.get('manufacturingInfo', {}).get('brand'),
    'componentCountry': product.get('manufacturingInfo', {}).get('componentCountry'),
    'specification': product.get('manufacturingInfo', {}).get('specification'),
    'skuId': product.get('skus', [{}])[0].get('skuId'),
    'msrpPrice': product.get('skus', [{}])[0].get('onlineOffer', {}).get('price', {}).get('finalPrice', {}).get('amount'),
    'generatedUPC': product.get('skus', [{}])[0].get('onlineOffer', {}).get('generatedUPC'),
    'gtin': product.get('skus', [{}])[0].get('onlineOffer', {}).get('gtin'),
    'inventoryStatus': product.get('skus', [{}])[0].get('onlineOffer', {}).get('inventory', {}).get('status'),
    'skuLogisticsLength': product.get('skus', [{}])[0].get('skuLogistics', {}).get('length', {}).get('value'),
    'skuLogisticsWidth': product.get('skus', [{}])[0].get('skuLogistics', {}).get('width', {}).get('value'),
    'skuLogisticsWeight': product.get('skus', [{}])[0].get('skuLogistics', {}).get('weight', {}).get('value'),
    'skuLogisticsHeight': product.get('skus', [{}])[0].get('skuLogistics', {}).get('height', {}).get('value')
}
# Printing key-value pairs as JSON format
print("{")
for i, (key, value) in enumerate(output.items()):
    # Convert non-string values to JSON-like format, handle None values
    value = str(value)
    value = re.sub(r'</?[^>]+>', '', value)
    value = re.sub(r'[\t\n]+', ' ', value).strip()
    value = re.sub(r'[\u00ae\u2122]|Core\u2122', '', value).strip()
    formatted_value = json.dumps(value) if value is not None else "null"
    # Add a comma after each item, except the last one
    comma = "," if i < len(output) - 1 else ""
    print(f'  "{key}": {formatted_value}{comma}')
print("}")

{
  "productId": "P03020955",
  "productType": "NORMALITEM",
  "productDescription": "HP \u2013 17.3\" Full HD IPS Laptop \u2013 Intel  i5 \u2013 8GB RAM \u2013 512GB SSD \u2013 2-Year HP Care Pack Warranty w/2-Year Accidental Damage Protection - Windows 11",
  "shortDescription": "Operating system: Windows 11 HomeProcessor: 12th Generation Intel  i5-1235UDisplay: 17.3-inch diagonal, FHD (1920 x 1080), IPS, anti-glareMemory: 8 GB DDR4-3200 MHz RAM (2 X 4 GB)",
  "longDescription": "Introducing the HP 17.3\" Full HD IPS Laptop, a powerful and reliable device designed to enhance your productivity and entertainment experience. With its impressive specifications and comprehensive warranty, this laptop is a great choice for both work and play. Featuring the latest Windows 11 Home operating system, you'll enjoy a seamless and intuitive user interface that enhances your productivity and creativity. The 12th Generation Intel Core i5-1235U processor ensures smooth performance and efficient mult